In [ ]:
!pip install transformers==3.2.0
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import statistics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Conv1D, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Conv1D, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import tensorflow_hub as hub


In [ ]:
from gensim.parsing import preprocessing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

nltk.download('punkt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import os
import pickle
import pandas as pd

In [ ]:
path_to_file = '/content/drive/MyDrive/test_all.csv'

df = pd.read_csv(path_to_file)


In [ ]:
df.head()

,id,hyperpartisan,bias,url,labeled-by,published-at,title,body
0,17,False,left-center,http://sfgate.com/politics/article/SAN-FRANCIS...,publisher,2004-07-29,SAN FRANCISCO / Head of Juvenile Probation Dep...,Chief juvenile probation officer Gwendolyn Tu...
1,23,True,right,https://thecollegefix.com/post/32204/,publisher,2017-04-20,"University leaders ban pro-life flag display, ...",Pro-life students at Wilfrid Laurier Universit...
2,29,True,left,https://wonkette.com/622815/donald-trump-get-y...,publisher,2017-09-12,"DONALD TRUMP, GET YOUR TINY PIGGY PERVERT HAND...",Time for Hillary Clinton to go back to Benghaz...
3,32,True,right,http://thegatewaypundit.com/2016/06/dirty-croo...,publisher,2016-06-05,DIRTY: Hillary Clinton Implies Trump is a Nazi...,Do you want to know why violent unhinged lefti...
4,37,False,right-center,https://cfr.org/report/bipartisan-work-plan,publisher,2015-01-12,A Bipartisan Work Plan,American workers continue to struggle with los...


In [ ]:
df_hyper_false = df[df.hyperpartisan == False]
df_hyper_true = df[df.hyperpartisan == True]

text_values_false = df_hyper_false.body.values
text_values_true = df_hyper_true.body.values
tragets_false = df_hyper_false.hyperpartisan.values
tragets_true = df_hyper_true.hyperpartisan.values

In [ ]:
dataset_size_div_by_2 = 50_000

text_values_false = text_values_false[:dataset_size_div_by_2]
tragets_false = tragets_false[:dataset_size_div_by_2]
#############################
text_values_true = text_values_true[:dataset_size_div_by_2]
tragets_true = tragets_true[:dataset_size_div_by_2]



text = np.concatenate((text_values_false, text_values_true), axis=0)
labels = np.concatenate((tragets_false, tragets_true), axis=0)


In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize


In [ ]:
sent_text = []
new_labels = []
for idx, txt in enumerate(text):
  if type(txt) == str:
    my_txt = txt.replace('\n', ' ').lower()
    # my_txt = preprocessing.remove_stopwords(my_txt)
    # my_txt = preprocessing.strip_tags(preprocessing.strip_numeric(my_txt))
    sents = sent_tokenize(txt)
    word_level_sents = []
    for sent in sents:
      word_level_sents.append(word_tokenize(sent))
    sent_text.append(word_level_sents)
    new_labels.append(labels[idx])



In [ ]:
size_sent = []
size_word = []
for txt in sent_text:
  size_sent.append(len(txt))
  for val in txt:
    size_word.append(len(val))

In [ ]:
np.mean(size_sent), np.mean(size_word)

(28.96020482662746, 28.483690329336063)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'
num_words = 35_000
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
to_pass_token = []
for idx, doc in enumerate(sent_text):
  for sent in doc:
    to_pass_token.append(' '.join(sent))

tokenizer.fit_on_texts(to_pass_token)


In [ ]:
transf_seq = []
max_seq_lenght = 30
max_sent_lenght = 20
for idx, doc in enumerate(sent_text):
  transf_seq.append([])
  for idx, sent in enumerate(doc):
    if idx == max_sent_lenght:
      break
    train_sequences = tokenizer.texts_to_sequences([' '.join(sent)])
    train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=max_seq_lenght)
    transf_seq[-1].append(train_padded[0])
  while len(transf_seq[-1]) != max_sent_lenght:
    train_padded = pad_sequences([[],[]], padding=pad_type, truncating=trunc_type, maxlen=max_seq_lenght)
    transf_seq[-1].append(train_padded[0])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1


In [ ]:
from tensorflow.keras import backend as K

def getModel(LSTM_CELL_SIZE):
    in_id = tf.keras.layers.Input((max_sent_lenght, max_seq_lenght), dtype='int32', name="input_shape")
    emb_words = tf.keras.layers.TimeDistributed(tf.keras.layers.Embedding(input_dim=num_words + 1, output_dim=32))(in_id)
    averaged_sents = tf.keras.layers.TimeDistributed(tf.keras.layers.GlobalAveragePooling1D())(emb_words)
    lstm_later, forward_h, forward_c= tf.keras.layers.LSTM(LSTM_CELL_SIZE, return_sequences=True, return_state=True)(averaged_sents)
    linear =  tf.keras.layers.Dense(LSTM_CELL_SIZE )(forward_h)
    attention = tf.keras.layers.dot([lstm_later, linear], axes=(-1))
    attention = tf.keras.layers.Activation('softmax',  name='attention_vec')(attention)
    attention = tf.keras.layers.RepeatVector(LSTM_CELL_SIZE)(attention)
    attention = tf.keras.layers.Permute([2, 1])(attention)
    sent_representation = tf.keras.layers.multiply([lstm_later, attention])
    sent_representation = tf.keras.layers.Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)
    sent_representation_final = tf.keras.layers.Concatenate()([sent_representation, forward_h])
    drop = tf.keras.layers.Dropout(0.2)(sent_representation)
    predictions = tf.keras.layers.Dense(2, activation='softmax')(drop)
    model = tf.keras.Model(inputs=in_id, outputs=predictions)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    return model



In [ ]:
model = getModel(100)

In [ ]:
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_shape (InputLayer)        [(None, 20, 30)]     0                                            
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 20, 30, 32)   1120032     input_shape[0][0]                
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 20, 32)       0           time_distributed_6[0][0]         
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 20, 100), (N 53200       time_distributed_7[0][0]         
_______________________________________________________________________________________

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(transf_seq, new_labels, test_size=0.1)
y_train = [to_categorical(i, num_classes=2) for i in y_train]
y_train= np.array(y_train)
y_test = [to_categorical(i, num_classes=2) for i in y_test]
y_test= np.array(y_test)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
X_train.shape ,  X_test.shape

((89988, 20, 30), (9999, 20, 30))

In [ ]:
# print(X_train_encoded[0].shape, X_train_encoded[0].dtype)
# print(X_train_encoded[1].shape, X_train_encoded[1].dtype)
# print(X_train_encoded[2].shape, X_train_encoded[2].dtype)
print(y_train.shape, y_train.dtype)
print(y_test.shape, y_test.dtype)

# model.fit(X_train_encoded, y_train, shuffle=True, epochs=3, batch_size=32, validation_data=(X_test_encoded, y_test),
#          sample_weight=sample_weights)

model.fit(X_train, y_train, shuffle=True, epochs=5, batch_size=64, validation_data=(X_test, y_test))

(89988, 2) float32
(9999, 2) float32
Epoch 1/5
1407/1407 [==============================] - 79s 56ms/step - loss: 0.4741 - acc: 0.7479 - val_loss: 0.3554 - val_acc: 0.8377
Epoch 2/5
1407/1407 [==============================] - 79s 56ms/step - loss: 0.3190 - acc: 0.8566 - val_loss: 0.3353 - val_acc: 0.8502
Epoch 3/5
1407/1407 [==============================] - 79s 56ms/step - loss: 0.2728 - acc: 0.8768 - val_loss: 0.3398 - val_acc: 0.8469
Epoch 4/5
1407/1407 [==============================] - 80s 57ms/step - loss: 0.2397 - acc: 0.8908 - val_loss: 0.3527 - val_acc: 0.8414
Epoch 5/5
1407/1407 [==============================] - 80s 57ms/step - loss: 0.2172 - acc: 0.9000 - val_loss: 0.3466 - val_acc: 0.8561


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
test_pred = model.predict(X_test_encoded)


NameError: ignored

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)

In [ ]:
pred_labels = np.reshape(pred_labels, (-1,))
test_labels = np.reshape(test_labels, (-1,))
print(classification_report(test_labels, pred_labels))

In [ ]:
# model.save_weights('/content/drive/My Drive/Colab Notebooks/model_weights/weights_6_dense_min/')

In [ ]:
#42 f1 on 2 min